# Encontros 13 e 14: Datathon

Importando as bibliotecas:

In [2]:
import sys
sys.path.append('..')

import socnet as sn
import numpy as np
from matplotlib import pyplot as plt

import plotly.plotly as py
import plotly.graph_objs as go

from scipy.stats.stats import pearsonr
from scipy.optimize import minimize

Definindo função que calcula *coreness*:

In [3]:
def cost(c, A, mask):
    row = np.matrix(c)

    C = np.multiply(row.transpose(), row)

    correlation, _ = pearsonr(C[~mask].flat, A[~mask].flat)

    return -correlation


def calculate_coreness(g):
    length = g.number_of_nodes()

    c = np.random.rand(length)

    A = sn.build_matrix(g)

    mask = np.eye(length, dtype=bool)

    bounds = [(0, 1) for _ in range(length)]

    result = minimize(cost, c, args=(A, mask), bounds=bounds)

    norm = 0

    for n in g.nodes:
        g.nodes[n]['coreness'] = result['x'][n]

        norm += g.nodes[n]['coreness'] ** 2

    for n in g.nodes:
        g.nodes[n]['coreness'] /= norm

Definindo função que calcula *PageRank*:

In [4]:
def equals(a, b):
    return abs(a - b) < 0.000000001


def calculate_pagerank(g):
    length = g.number_of_nodes()

    k = 10
    scale = 0.8
    residue = (1 - scale) / length

    R = sn.build_matrix(g)

    for n in g.nodes:
        total = np.sum(R[n,])

        if equals(total, 0):
            R[n, n] = 1
        else:
            R[n,] /= total

    R = scale * R + residue

    Rt = R.transpose()

    rank = 1 / length

    r = np.full((length, 1), rank)

    for _ in range(k):
        r = Rt.dot(r)

    for n in g.nodes:
        g.nodes[n]['pagerank'] = r[n, 0]

Lista de todos os códigos de países, para facilitar a iteração pelos arquivos.

Não esqueça que também existem os arquivos `<ano>-all-countries.gml` que representam a soma dos países.

In [5]:
country_codes = [
    'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHN', 'CYP',
    'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR',
    'GRC', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR',
    'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'POL', 'PRT',
    'ROM', 'RUS', 'SVK', 'SVN', 'SWE', 'TUR', 'TWN', 'USA',
]

Lista de todos os códigos de indústrias, para facilitar a iteração pelos arquivos.

Não esqueça que também existem os arquivos `<ano>-all-industries.gml` que representam a soma das indústrias.

In [6]:
industry_codes = [
     'c1',  'c2',  'c3',  'c4',  'c5',  'c6',  'c7',
     'c8',  'c9', 'c10', 'c11', 'c12', 'c13', 'c14',
    'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21',
    'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28',
    'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35',
]

In [7]:
file = "grafos/{}-{}.gml"

country_graphs = {}

count = 1

for country in country_codes:
    year = 1995
    country_graphs[country] = {}
    while year <= 2007:
        country_graphs[country][year] = sn.load_graph(file.format(year, country))
        year+=1

FileNotFoundError: [Errno 2] No such file or directory: 'grafos/1995-AUS.gml'

In [ ]:
file = "grafos/{}-{}.gml"

ind_graphs = {}


for ind in industry_codes:
    year = 1995
    ind_graphs[ind] = {}
    while year <= 2007:
        ind_graphs[ind][year] = sn.load_graph(file.format(year, ind))
        year+=1

In [ ]:
for i in country_graphs:
    for year in country_graphs[i]:
        print(i,year)
        calculate_coreness(country_graphs[i][year])
    

for i in country_graphs:
    for year in country_graphs[i]:
        print(i,year)
        calculate_pagerank(country_graphs[i][year])
    
for i in ind_graphs:
    for year in ind_graphs[i]:
        print(i,year)
        calculate_coreness(ind_graphs[i][year])
    
for i in ind_graphs:
    for year in ind_graphs[i]:
        print(i,year)
        calculate_pagerank(ind_graphs[i][year])
    

In [ ]:
all_countries = {}

year = 1995
while year <= 2007:
    all_countries [year] = sn.load_graph(file.format(year, "all-countries"))
    year+=1

In [ ]:
all_industries = {}

year = 1995
while year <= 2007:
    all_industries [year] = sn.load_graph(file.format(year, "all-industries"))
    year+=1

In [ ]:
lista = []
year = 1998
g = all_industries[year]
for n in g:
    lista.append([g.node[n]['gdp'], ])

In [ ]:
plt.plot(sorted(lista))
plt.show()